# import libraries

In [12]:
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
plt.style.use("default")
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_percentage_error,mean_absolute_error
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.multioutput import RegressorChain
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import LinearRegression,Lasso,Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import RandomizedSearchCV
from math import sqrt
from sklearn import linear_model
from sklearn.neighbors import NearestCentroid,KNeighborsRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import RandomForestRegressor
from utils import feature_impute, feature_impute_exiting, rf_fill, select_feature, plot_SSP
import random
from sklearn.model_selection import cross_val_score, KFold
import time
import datetime

In [13]:
df_all=pd.read_excel("data.xlsx")
unseen=pd.read_excel("unseen_data_fingernet.xlsx")
# df_all.columns
Yc=['RP','RR']
y=df_all[Yc]
x=df_all.drop(Yc,axis=1)
y_unseen = unseen[Yc]
x_unseen = unseen.drop(Yc,axis=1)

In [14]:
need_delete=['Surface roughness']
need_statistic_filling=['Permeability','Rejection/Selectivity','Amine concentration','Chloride concentration']
need_model_filling=['Water contact angle','Solute concentration','NP size','Solute molecular weight']
needed_model_filling=['Amine concentration','Chloride concentration','Chloride concentration','Rejection/Selectivity']
x.drop(need_delete,axis=1,inplace=True)
x_unseen.drop(need_delete,axis=1,inplace=True)
filling=x[need_statistic_filling].mean()
x[need_statistic_filling]=x[need_statistic_filling].fillna(dict(filling), inplace=False) ##
x_unseen[need_statistic_filling]=x_unseen[need_statistic_filling].fillna(dict(filling), inplace=False) ##

In [ ]:
model=[]
for target, use_f in zip(need_model_filling,needed_model_filling):
    x, m=feature_impute(x, target, use_f)
    model.append(m)
    rf_fill(x, target)
for m, target, use_f in zip(model, need_model_filling, needed_model_filling):
    x_unseen=feature_impute_exiting(x_unseen, target, use_f, model=m)
    rf_fill(x_unseen, target)

In [16]:
df_all = pd.concat([x,x_unseen],axis=0)
col = ['Logp', 'Tpsa', 'Qed', 'Hba', 'Hbd', 'Rob']
col1 = ['Solvent_logp', 'Solvent_tpsa', 'Solvent_qed', 'Solvent_hba', 'Solvent_hbd', 'Solvent_rob']
col2 = ['Solute_logp', 'Solute_tpsa', 'Solute_qed', 'Solute_hba', 'Solute_hbd', 'Solute_rob']
smiles=pd.read_excel("smiles.xlsx")
solute = []
for i in range(df_all.shape[0]):
    if df_all.iloc[i]['Solute_type'] == 'AO':
        solute.append(smiles.loc[10][col])
    elif df_all.iloc[i]['Solute_type'] == 'BTB':
        solute.append(smiles.loc[11][col])
    elif df_all.iloc[i]['Solute_type'] == 'CV':
        solute.append(smiles.loc[12][col])
    elif df_all.iloc[i]['Solute_type'] == 'MB':
        solute.append(smiles.loc[13][col])
    elif df_all.iloc[i]['Solute_type'] == 'MO':
        solute.append(smiles.loc[14][col])
    elif df_all.iloc[i]['Solute_type'] == 'PEG':
        solute.append(smiles.loc[15][col])
    elif df_all.iloc[i]['Solute_type'] == 'RB':
        solute.append(smiles.loc[16][col])
    elif df_all.iloc[i]['Solute_type'] == 'RDB':
        solute.append(smiles.loc[17][col])
    elif df_all.iloc[i]['Solute_type'] == 'tetracycline':
        solute.append(smiles.loc[18][col])
    elif df_all.iloc[i]['Solute_type'] == 'oligomer':
        solute.append(smiles.loc[19][col])
    elif df_all.iloc[i]['Solute_type'] == 'SY':
        solute.append(smiles.loc[20][col])
    elif df_all.iloc[i]['Solute_type'] == 'EB':
        solute.append(smiles.loc[21][col])
    elif df_all.iloc[i]['Solute_type'] == 'RBBR':
        solute.append(smiles.loc[22][col])
    elif df_all.iloc[i]['Solute_type'] == 'SO':
        solute.append(smiles.loc[23][col])
df_all[col2] = solute

In [17]:
# For data after molecular characterization of solvents and solutes,
# solvent type and solute type should be removed to prevent redundancy of information.
x=df_all.copy()
x.drop(['Solute_type'], axis=1, inplace=True) #, 'Solvent_type'
x=pd.get_dummies(x)
is_delete = []
for c in col2: # +col1
    if x[0:653][c].max() == 0:
        is_delete.append(c)
x.drop(columns=is_delete, axis=1, inplace=True)
# Use the minimum and maximum values of the former
min = x.iloc[:653,:].min()
max = x.iloc[:653,:].max()
# Make changes to the entire dataset
x = (x - min) / max
x_unseen=x.iloc[653:, :]
x=x.iloc[:653, :]
# x, x_unseen = select_feature(x, y, x_unseen)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, shuffle = True, random_state = 60)

In [19]:
tree_model=[
    RandomForestRegressor(n_estimators=100, max_leaf_nodes=10, max_features=100, max_depth=15, random_state=42),
    XGBRegressor(subsample=0.7,reg_lambda=1,reg_alpha=0.01,n_estimators=1000,min_child_weight=1,max_depth=15,
    learning_rate=0.05,gamma=0.1,colsample_bytree=0.7,random_state=42),
    AdaBoostRegressor(n_estimators=1000,learning_rate=1.3,random_state=42),
    GradientBoostingRegressor(learning_rate=0.01,n_estimators=100,subsample=0.5,random_state=42, max_features=100),
    LGBMRegressor(max_depth = 15,learning_rate=0.03, n_estimators=100, seed = 42),
    DecisionTreeRegressor(max_leaf_nodes=10, max_features=100, max_depth=15, random_state=42),
    BaggingRegressor(n_estimators=100,max_samples=0.70, max_features=0.75, bootstrap=True, random_state=42)
            ]

untree_model=[
    Lasso(alpha=0.0),
    Ridge(),
    linear_model.ElasticNet(alpha=0.1, l1_ratio=0.1),
    linear_model.BayesianRidge(n_iter=500, tol=0.006, alpha_1=3e-06, alpha_2=3e-06, lambda_1=2e-06, lambda_2=2e-06),
    KNeighborsRegressor(n_neighbors=4),
    SVR(kernel='linear')]

model_name=['RF','XGB','AdaB','GradB','LGBM','DT','Bag','Lasso','Ridge','ENet','Bayes','KNN','SVM']

In [20]:
kfold = KFold(n_splits = 5, shuffle=True, random_state=6)
results=[]
def get_ml_multioutput_predict(m):
    scores1 = []
    scores2 = []
    t_scores1 = []
    t_scores2 = []

    rmse1 = []
    rmse2 = []
    t_rmse1 = []
    t_rmse2 = []

    mape1 = []
    mape2 = []
    t_mape1 = []
    t_mape2 = []

    mae1 = []
    mae2 = []
    t_mae1 = []
    t_mae2 = []
    for train, test in kfold.split(x_train, y_train):
        model = MultiOutputRegressor(m)
        model.fit(x_train.iloc[train], y_train.iloc[train])
        y_train_pred=model.predict(x_train.iloc[train])
        y_val_pred=model.predict(x_train.iloc[test])
        scores1.append(r2_score(y_train.iloc[test].values[:,0],y_val_pred[:,0]))
        scores2.append(r2_score(y_train.iloc[test].values[:,1],y_val_pred[:,1]))
        t_scores1.append(r2_score(y_train.iloc[train].values[:,0],y_train_pred[:,0]))
        t_scores2.append(r2_score(y_train.iloc[train].values[:,1],y_train_pred[:,1]))
        rmse1.append(sqrt(mean_squared_error(y_train.iloc[test].values[:,0],y_val_pred[:,0])))
        rmse2.append(sqrt(mean_squared_error(y_train.iloc[test].values[:,1],y_val_pred[:,1])))
        t_rmse1.append(sqrt(mean_squared_error(y_train.iloc[train].values[:,0],y_train_pred[:,0])))
        t_rmse2.append(sqrt(mean_squared_error(y_train.iloc[train].values[:,1],y_train_pred[:,1])))
        mape1.append(mean_absolute_percentage_error(y_train.iloc[test].values[:,0],y_val_pred[:,0]))
        mape2.append(mean_absolute_percentage_error(y_train.iloc[test].values[:,1],y_val_pred[:,1]))
        t_mape1.append(mean_absolute_percentage_error(y_train.iloc[train].values[:,0],y_train_pred[:,0]))
        t_mape2.append(mean_absolute_percentage_error(y_train.iloc[train].values[:,1],y_train_pred[:,1]))
        mae1.append(mean_absolute_error(y_train.iloc[test].values[:,0],y_val_pred[:,0]))
        mae2.append(mean_absolute_error(y_train.iloc[test].values[:,1],y_val_pred[:,1]))
        t_mae1.append(mean_absolute_error(y_train.iloc[train].values[:,0],y_train_pred[:,0]))
        t_mae2.append(mean_absolute_error(y_train.iloc[train].values[:,1],y_train_pred[:,1]))
    result=[]
    result.append(np.array(scores1).mean())
    result.append(np.array(scores1).std())
    result.append(np.array(scores2).mean())
    result.append(np.array(scores2).std())
    result.append(np.array(rmse1).mean())
    result.append(np.array(rmse1).std())
    result.append(np.array(rmse2).mean())
    result.append(np.array(rmse2).std())
    result.append(np.array(mape1).mean())
    result.append(np.array(mape1).std())
    result.append(np.array(mape2).mean())
    result.append(np.array(mape2).std())
    result.append(np.array(mae1).mean())
    result.append(np.array(mae1).std())
    result.append(np.array(mae2).mean())
    result.append(np.array(mae2).std())
    result.append(np.array(t_scores1).mean())
    result.append(np.array(t_scores1).std())
    result.append(np.array(t_scores2).mean())
    result.append(np.array(t_scores2).std())
    result.append(np.array(t_rmse1).mean())
    result.append(np.array(t_rmse1).std())
    result.append(np.array(t_rmse2).mean())
    result.append(np.array(t_rmse2).std())
    result.append(np.array(t_mape1).mean())
    result.append(np.array(t_mape1).std())
    result.append(np.array(t_mape2).mean())
    result.append(np.array(t_mape2).std())
    result.append(np.array(t_mae1).mean())
    result.append(np.array(t_mae1).std())
    result.append(np.array(t_mae2).mean())
    result.append(np.array(t_mae2).std())
    results.append(result)
    print('r2:')
    print(f'train_mean1:{np.array(t_scores1).mean()}, train_std1:{np.array(t_scores1).std()}, \
            train_mean2:{np.array(t_scores2).mean()},train_std2:{np.array(t_scores2).std()},\
            test_mean1:{np.array(scores1).mean()}, test_std1:{np.array(scores1).std()},\
            test_mean2:{np.array(scores2).mean()}, test_std2:{np.array(scores2).std()}')
    print('\n')
    print('rmse:')
    print(f'train_mean1:{np.array(t_rmse1).mean()}, train_std1:{np.array(t_rmse1).std()}, \
            train_mean2:{np.array(t_rmse2).mean()},train_std2:{np.array(t_rmse2).std()},\
            test_mean1:{np.array(rmse1).mean()}, test_std1:{np.array(rmse1).std()},\
            test_mean2:{np.array(rmse2).mean()}, test_std2:{np.array(rmse2).std()}')
    print('\n')
    print('mape:')
    print(f'train_mean1:{np.array(t_mape1).mean()}, train_std1:{np.array(t_mape1).std()}, \
            train_mean2:{np.array(t_mape2).mean()},train_std2:{np.array(t_mape2).std()},\
            test_mean1:{np.array(mape1).mean()}, test_std1:{np.array(mape1).std()},\
            test_mean2:{np.array(mape2).mean()}, test_std2:{np.array(mape2).std()}')
    print('\n')
    print('mae:')
    print(f'train_mean1:{np.array(t_mae1).mean()}, train_std1:{np.array(t_mae1).std()}, \
            train_mean2:{np.array(t_mae2).mean()},train_std2:{np.array(t_mae2).std()},\
            test_mean1:{np.array(mae1).mean()}, test_std1:{np.array(mae1).std()},\
            test_mean2:{np.array(mae2).mean()}, test_std2:{np.array(mae2).std()}')

In [ ]:
for m in tree_model+untree_model:
    print(m)
    print('\n')
    print(get_ml_multioutput_predict(m))
# pd.DataFrame(np.array(results)).to_excel('ml_cross_validation.xlsx')

In [11]:
def train(m, seed=6):
    i=0
    random.seed(seed)
    np.random.seed(seed)
    try:
        m.set_params(random_state=seed)
    except:
        i=i+1
    try:
        m.set_params(seed=seed)
    except:
        i=i+1
    model = MultiOutputRegressor(m)
    model.fit(x_train, y_train)
    y_predt = model.predict(x_train)
    y_pred = model.predict(x_test)
    y_predu = model.predict(x_unseen)
#     print(y_predu)
    r2_test_rwp.append(r2_score(y_test.values[:, 0],y_pred[:, 0]))
    r2_test_rsp.append(r2_score(y_test.values[:, 1],y_pred[:, 1]))
    rmse_test_rwp.append(sqrt(mean_squared_error(y_test.values[:,0],y_pred[:,0])))
    rmse_test_rsp.append(sqrt(mean_squared_error(y_test.values[:,1],y_pred[:,1])))
    mape_test_rwp.append(mean_absolute_percentage_error(y_test.values[:,0],y_pred[:,0]))
    mape_test_rsp.append(mean_absolute_percentage_error(y_test.values[:,1],y_pred[:,1]))
    mae_test_rwp.append(mean_absolute_error(y_test.values[:,0],y_pred[:,0]))
    mae_test_rsp.append(mean_absolute_error(y_test.values[:,1],y_pred[:,1]))
    
    r2_train_rwp.append(r2_score(y_train.values[:, 0],y_predt[:, 0]))
    r2_train_rsp.append(r2_score(y_train.values[:, 1],y_predt[:, 1]))
    rmse_train_rwp.append(sqrt(mean_squared_error(y_train.values[:, 0],y_predt[:, 0])))
    rmse_train_rsp.append(sqrt(mean_squared_error(y_train.values[:, 1],y_predt[:, 1])))
    mape_train_rwp.append(mean_absolute_percentage_error(y_train.values[:, 0],y_predt[:, 0]))
    mape_train_rsp.append(mean_absolute_percentage_error(y_train.values[:, 1],y_predt[:, 1]))
    mae_train_rwp.append(mean_absolute_error(y_train.values[:, 0],y_predt[:, 0]))
    mae_train_rsp.append(mean_absolute_error(y_train.values[:, 1],y_predt[:, 1]))
    print('R2:',r2_score(y_test.values[:, 0],y_pred[:, 0]), r2_score(y_test.values[:, 1],y_pred[:, 1]))
    print("rmse:",sqrt(mean_squared_error(y_test.values[:,0],y_pred[:,0])), sqrt(mean_squared_error(y_test.values[:,1],y_pred[:,1])))
    print("mape:",mean_absolute_percentage_error(y_test.values[:,0],y_pred[:,0]), mean_absolute_percentage_error(y_test.values[:,1],y_pred[:,1]))
    print("mae:",mean_absolute_error(y_test.values[:,0],y_pred[:,0]), mean_absolute_error(y_test.values[:,1],y_pred[:,1]))
    return y_predu

In [ ]:
import time
import datetime
import random
starttime = datetime.datetime.now()
results=[]
for m_name, m in zip(model_name, tree_model+untree_model):
    result=[]
    print(m_name)
    r2_test_rwp=[]
    r2_test_rsp=[]
    rmse_test_rwp=[]
    rmse_test_rsp=[]
    mape_test_rwp=[]
    mape_test_rsp=[]
    mae_test_rwp=[]
    mae_test_rsp=[]
    r2_train_rwp=[]
    r2_train_rsp=[]
    rmse_train_rwp=[]
    rmse_train_rsp=[]
    mape_train_rwp=[]
    mape_train_rsp=[]
    mae_train_rwp=[]
    mae_train_rsp=[]
    y_unseen_sum=np.zeros(y_unseen.shape)
    for n, i in enumerate([random.choice([i for i in range(1000)]) for i in range(100)]):
        y_unseen_temp = train(m, seed=i)
        y_unseen_sum = y_unseen_sum+y_unseen_temp
        endtime = datetime.datetime.now()
        print(m_name+f'_随机种子测量完成，目前已耗时{(endtime - starttime).seconds} s')
    for i in [
            r2_test_rwp,r2_test_rsp,rmse_test_rwp,rmse_test_rsp,mape_test_rwp,mape_test_rsp,mae_test_rwp,mae_test_rsp, 
            r2_train_rwp,r2_train_rsp,rmse_train_rwp,rmse_train_rsp,mape_train_rwp,mape_train_rsp,mae_train_rwp,mae_train_rsp
                ]:
        result.append(np.array(i).mean())
        result.append(np.array(i).std())
    results.append(result)
    r2_train_rwp,r2_train_rsp,rmse_train_rwp,rmse_train_rsp,mape_train_rwp,mape_train_rsp,\
    mae_train_rwp,mae_train_rsp,r2_test_rwp,r2_test_rsp,rmse_test_rwp,rmse_test_rsp,mape_test_rwp,\
    mape_test_rsp,mae_test_rwp,mae_test_rsp = pd.DataFrame(r2_train_rwp),pd.DataFrame(r2_train_rsp),\
    pd.DataFrame(rmse_train_rwp),pd.DataFrame(rmse_train_rsp),pd.DataFrame(mape_train_rwp),\
    pd.DataFrame(mape_train_rsp),pd.DataFrame(mae_train_rwp),pd.DataFrame(mae_train_rsp),\
    pd.DataFrame(r2_test_rwp),pd.DataFrame(r2_test_rsp),pd.DataFrame(rmse_test_rwp),\
    pd.DataFrame(rmse_test_rsp),pd.DataFrame(mape_test_rwp),pd.DataFrame(mape_test_rsp),\
    pd.DataFrame(mae_test_rwp),pd.DataFrame(mae_test_rsp)
    ml_metrics=pd.concat([r2_train_rwp,r2_train_rsp,rmse_train_rwp,rmse_train_rsp,mape_train_rwp,mape_train_rsp,mae_train_rwp,mae_train_rsp,
            r2_test_rwp,r2_test_rsp,rmse_test_rwp,rmse_test_rsp,mape_test_rwp,mape_test_rsp,mae_test_rwp,mae_test_rsp],axis=1)
    ml_metrics.to_excel(m_name+'_metrics.xlsx')
    y_unseen_sum_last=y_unseen_sum.copy()
    y_unseen_sum_last=y_unseen_sum_last/100
#     print('R2:',r2_score(y_unseen.values[:, 0],y_unseen_sum_last[:, 0]), r2_score(y_unseen.values[:, 1],y_unseen_sum_last[:, 1]))
#     print("rmse:",sqrt(mean_squared_error(y_unseen.values[:,0],y_unseen_sum_last[:,0])), sqrt(mean_squared_error(y_unseen.values[:,1],y_unseen_sum_last[:,1])))
#     print("mape:",mean_absolute_percentage_error(y_unseen.values[:,0],y_unseen_sum_last[:,0]), mean_absolute_percentage_error(y_unseen.values[:,1],y_unseen_sum_last[:,1]))
#     print("mae:",mean_absolute_error(y_unseen.values[:,0],y_unseen_sum_last[:,0]), mean_absolute_error(y_unseen.values[:,1],y_unseen_sum_last[:,1]))
    pd.DataFrame(y_unseen_sum_last).to_excel(m_name+'_unseen.xlsx')
endtime = datetime.datetime.now()
print(f'The total time taken for this repeat measurement：{(endtime - starttime).seconds}')

In [14]:
# np.array(results).shape
# pd.DataFrame(np.array(results)).to_excel('ml_performance.xlsx')